In [2]:
!pwd

/home/monoids/notebooks/CommutativeMonoids/ClassCSemigroup


In [3]:
import Auxiliares as aux

import PyNormaliz
from PyNormaliz import *
import numpy
from numpy import *
import itertools
from scipy.spatial import ConvexHull


import sys
sys.path.insert(0,'./CommutativeMonoids/Class/')
sys.path.insert(0,'./CommutativeMonoids/ClassAffine')
import integerSmithNormalFormAndApplications
from integerSmithNormalFormAndApplications import *
import AffineSemigroup
from AffineSemigroup import *
import auxiliars
from auxiliars import *


In [3]:
def DeleteCSemigruposDuplicates(v):
    limpio = []
    aux = []
    for x in v:
        if x.generadores not in aux:
            aux.append(x.generadores)
            limpio.append(x)
    return limpio

In [6]:
class Csemigroup:
    gaps = None
    pf = None
    def __init__(self,generadores):
        self.generadores = AffineSemigroup(generadores,input_type="generators").getMSG()
        self.generadores.sort(key=lambda row: row[1:])
        if not aux.IsCsemigroup(self.generadores):
            raise Exception("The set do not form a C-Semigroup")
        self.gaps = aux.ComputeGaps(generadores)
    def GetGenerators(self):
        return self.generadores
    def GetGaps(self):
        return self.gaps
    # This function compute the Pseudo-Frobenius elements of a C-semigroup.
    # INPUT:
    #   - gen: generadores del C-semigrupo.
    #   - gap: conjunto de huecos del C-semigrupo.
    # OUTPUT:
    #   - conjunto de pseudo-frobenius.
    def ComputePseudoFrobenius(self):
        if self.pf != None:
            return self.pf
        cs = AffineSemigroup(self.generadores,input_type="generators")
        pf = []
        for x in self.gaps:
            ispf = True
            for y in self.generadores:
                if not cs.belongs([x[i]+y[i] for i in range(len(x))]):
                    ispf = False
            if ispf:
                pf.append(x)
        self.pf = pf
        return pf
    
    # Esta función clasifica en irreducible o no a un C-semigrupo.
    # INPUT:
    #   - pf: Pseudo-Frobenius.
    # OUTPUT:
    #   True/False
    def IsIrreducible(self):
        if self.pf == None:
            self.ComputePseudoFrobenius()
        if (len(self.pf) == 1) or (len(self.pf) == 2 and len(aux.DeleteHalves(self.pf)) == 1):
            return True
        return False
    
    def DecomposeCSemigroup(self):
        if self.IsIrreducible():
            return [self]
        candidatos = []
        pf = aux.DeleteHalves(self.pf)
        for x in pf:
            cs = AffineSemigroup(self.generadores+[x],input_type="generators")
            candidatos.append(Csemigroup(cs.getMSG()))
            #candidatos.append(cs.getMSG())
        return candidatos
    
    def DecomposeIrreducible(self):
        candidatos = [self]
        irreducibles = []
        #control = 0
        #while candidatos != [] and control <= 100:
        while candidatos != []:
            auxiliar = []
            for x in candidatos:
                auxiliar = auxiliar + x.DecomposeCSemigroup()
            auxiliar = DeleteCSemigruposDuplicates(auxiliar)
            for x in auxiliar:
                if x.IsIrreducible():
                    irreducibles.append(x)
                    auxiliar.remove(x)
            candidatos = auxiliar
            #control = control+1
        #print([x.generadores for x in irreducibles])
        #print()
        #for x in irreducibles:
        #    for y in irreducibles:
        #        if y.IsSubsemigroup(x) and x != y:
        #            irreducibles.remove(y)
        #return irreducibles
        irreduciblesNecesarios = []
        for x in irreducibles:
            necesario = True
            for y in irreducibles:
                if x.IsSubsemigroup(y) and x != y:
                    necesario = False
                    break
            if necesario:
                irreduciblesNecesarios.append(x)
        return irreduciblesNecesarios
        
            
    def __belongsByGens(self,x, gens):
        '''
        x is a numpy.array
        gens is a list of list whose size is equal to x length
        Returns True in case x = a_1*gens[0]+...+a_·*gens[·]
        '''
        if gens==[]:
            return False
        if all([e==0 for e in x]):
            return True
        if any([e<0 for e in x]):
            return False
        if belongsByGens(x-np.array(gens[0]), gens):
            return True
        elif len(gens)==1: 
            return False
        else: 
            return belongsByGens(x, gens[-(len(gens)-1):])
        
    def Belongs(self,x):
        return self.__belongsByGens(x,self.generadores)
    
    # Nos devuelve True si other es subsemigrupo de self
    def IsSubsemigroup(self,otherSemigroup):
        if all([x in otherSemigroup.gaps for x in self.gaps]):
            return True
        else:
            return False

In [7]:
cs = Csemigroup([[3, 1], [7, 1], [8, 2], [9, 1], [9, 2], [10, 1], [10, 3], [10, 4], [11, 2], [13, 1], [14, 5], [15, 6], [17, 7], [19, 8], [21, 2], [24, 2], [25, 2], [26, 11], [27, 2], [28, 2], [29, 2], [44, 3], [45, 3], [61, 4], [62, 4], [76, 5], [77, 5], [93, 6], [108, 7], [43, 3], [59, 4], [60, 4], [92, 6], [42, 18], [56, 24], [63, 27], [70, 30]])

[[3, 1], [7, 1], [9, 1], [13, 1], [10, 1], [9, 2], [11, 2], [24, 2], [28, 2], [8, 2], [21, 2], [27, 2], [25, 2], [29, 2], [10, 3], [43, 3], [45, 3], [44, 3], [60, 4], [62, 4], [59, 4], [61, 4], [10, 4], [77, 5], [76, 5], [14, 5], [92, 6], [93, 6], [15, 6], [108, 7], [17, 7], [19, 8], [26, 11], [42, 18], [56, 24], [63, 27], [70, 30]]


KeyboardInterrupt: 

In [5]:
sg1 = Csemigroup([[2,2],[3,0],[3,1],[3,2],[3,3],[4,0],[4,1],[4,2],[4,3],[5,0],[5,1],[5,2],[5,3]])

In [6]:
#sg1.generadores

In [95]:
sg1.GetGaps()

[[1, 0], [1, 1], [2, 0], [2, 1]]

In [96]:
desc  = sg1.DecomposeIrreducible()
for x in desc:
    print("->", x.generadores,"-",x.gaps)

[[[2, 0], [3, 0], [1, 1], [3, 2]], [[1, 0], [3, 1], [2, 2], [3, 3]], [[4, 0], [3, 0], [5, 0], [2, 1], [3, 1], [1, 1]], [[2, 0], [3, 0], [1, 1], [2, 1]], [[1, 0], [2, 1], [2, 2], [3, 3]]]

-> [[2, 0], [3, 0], [1, 1], [3, 2]] - [[1, 0], [2, 1]]
-> [[1, 0], [3, 1], [2, 2], [3, 3]] - [[1, 1], [2, 1]]
-> [[4, 0], [3, 0], [5, 0], [2, 1], [3, 1], [1, 1]] - [[1, 0], [2, 0]]


In [97]:
cd1 = Csemigroup([[2, 0], [3, 0], [1, 1], [3, 2]])
cd2 = Csemigroup([[1, 0], [3, 1], [2, 2], [3, 3]])
cd3 = Csemigroup([[4, 0], [3, 0], [5, 0], [2, 1], [3, 1], [1, 1]])
cd4 = Csemigroup([[2, 0], [3, 0], [1, 1], [2, 1]])
cd5 = Csemigroup([[1, 0], [2, 1], [2, 2], [3, 3]])
semigrupos = [cd1,cd2,cd3,cd4,cd5]
for x in semigrupos:
    print("->",x.generadores, " - ",x.gaps)
print()
for x in semigrupos:
    for y in semigrupos:
        if y.IsSubsemigroup(x) and x != y:
            print(x.gaps,"+++",y.gaps)
            semigrupos.remove(y)
for x in semigrupos:
    print("--->",x.generadores, " - ",x.gaps)

-> [[2, 0], [3, 0], [1, 1], [3, 2]]  -  [[1, 0], [2, 1]]
-> [[1, 0], [3, 1], [2, 2], [3, 3]]  -  [[1, 1], [2, 1]]
-> [[4, 0], [3, 0], [5, 0], [2, 1], [3, 1], [1, 1]]  -  [[1, 0], [2, 0]]
-> [[2, 0], [3, 0], [1, 1], [2, 1]]  -  [[1, 0]]
-> [[1, 0], [2, 1], [2, 2], [3, 3]]  -  [[1, 1]]

[[1, 0], [2, 1]] +++ [[1, 0]]
[[1, 1], [2, 1]] +++ [[1, 1]]
---> [[2, 0], [3, 0], [1, 1], [3, 2]]  -  [[1, 0], [2, 1]]
---> [[1, 0], [3, 1], [2, 2], [3, 3]]  -  [[1, 1], [2, 1]]
---> [[4, 0], [3, 0], [5, 0], [2, 1], [3, 1], [1, 1]]  -  [[1, 0], [2, 0]]


In [98]:
cd4.IsSubsemigroup(cd1)

True

In [99]:
#sg1.Belongs([1,0])

In [6]:
sg2 = Csemigroup([[2,0,0],[4,2,4],[0,1,0],[3,0,0],[6,3,6],[3,1,1],[4,1,1],[3,1,2],[1,1,0],[3,2,3],[1,2,1]])

In [9]:
sg2.pf

In [10]:
sg2.gaps

[[1, 0, 0], [1, 1, 1], [2, 1, 1], [2, 1, 2], [2, 2, 1], [2, 2, 2], [2, 3, 2], 
[4, 1, 2], [4, 2, 3], [5, 2, 4], [5, 3, 5], [8, 4, 7]]

In [11]:
sg2.ComputePseudoFrobenius()

[[2, 2, 1], [2, 3, 2], [4, 1, 2], [8, 4, 7]]

In [103]:
desc2  = sg2.DecomposeIrreducible()

[[[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [4, 1, 1], [3, 1, 1], [3, 1, 2], [4, 1, 2], [1, 2, 1], [2, 2, 1], [2, 2, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]], [[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [3, 1, 1], [3, 1, 2], [4, 1, 2], [1, 2, 1], [2, 2, 2], [3, 2, 3], [4, 2, 4], [5, 2, 4], [5, 3, 5], [6, 3, 6]], [[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [4, 1, 1], [3, 1, 1], [2, 1, 2], [3, 1, 2], [1, 2, 1], [2, 2, 1], [3, 2, 3]], [[1, 0, 0], [0, 1, 0], [2, 1, 1], [3, 1, 2], [1, 2, 1], [3, 2, 3], [4, 2, 4], [5, 3, 5], [6, 3, 6]], [[1, 0, 0], [0, 1, 0], [2, 1, 1], [3, 1, 2], [1, 2, 1], [2, 2, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]], [[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [1, 1, 1], [3, 1, 2], [4, 1, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]], [[1, 0, 0], [0, 1, 0], [2, 1, 1], [3, 1, 2], [1, 2, 1], [3, 2, 3], [4, 2, 4], [2, 3, 2], [5, 3, 5], [6, 3, 6]], [[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [1, 1, 1], [3, 1, 2], [3, 2, 3], [4, 2, 4], [5, 2, 4], [6, 3

In [12]:
intermedio = [[[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [4, 1, 1], [3, 1, 1], [3, 1, 2], [4, 1, 2], [1, 2, 1], [2, 2, 1], [2, 2, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]], [[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [3, 1, 1], [3, 1, 2], [4, 1, 2], [1, 2, 1], [2, 2, 2], [3, 2, 3], [4, 2, 4], [5, 2, 4], [5, 3, 5], [6, 3, 6]], [[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [4, 1, 1], [3, 1, 1], [2, 1, 2], [3, 1, 2], [1, 2, 1], [2, 2, 1], [3, 2, 3]], [[1, 0, 0], [0, 1, 0], [2, 1, 1], [3, 1, 2], [1, 2, 1], [3, 2, 3], [4, 2, 4], [5, 3, 5], [6, 3, 6]], [[1, 0, 0], [0, 1, 0], [2, 1, 1], [3, 1, 2], [1, 2, 1], [2, 2, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]], [[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [1, 1, 1], [3, 1, 2], [4, 1, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]], [[1, 0, 0], [0, 1, 0], [2, 1, 1], [3, 1, 2], [1, 2, 1], [3, 2, 3], [4, 2, 4], [2, 3, 2], [5, 3, 5], [6, 3, 6]], [[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [1, 1, 1], [3, 1, 2], [3, 2, 3], [4, 2, 4], [5, 2, 4], [6, 3, 6]], [[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [1, 1, 1], [2, 1, 2], [3, 1, 2]], [[1, 0, 0], [0, 1, 0], [1, 1, 1], [3, 1, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]], [[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [1, 1, 1], [2, 1, 2], [3, 1, 2]], [[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [4, 1, 1], [3, 1, 1], [2, 1, 2], [3, 1, 2], [1, 2, 1], [3, 2, 3], [4, 2, 3]], [[1, 0, 0], [0, 1, 0], [2, 1, 1], [2, 1, 2], [1, 2, 1], [3, 2, 3]], [[1, 0, 0], [0, 1, 0], [3, 1, 1], [2, 1, 2], [1, 2, 1], [3, 2, 3]]]

In [13]:
len(intermedio)

14

In [14]:
len(desc2)

NameError: name 'desc2' is not defined

In [119]:
hue = []
for x in desc2:
    hue = hue + x.gaps
    print(x.gaps)
DeleteDuplicates(hue)
print()
for x in desc2:
    print(x.generadores)

[[1, 0, 0], [1, 1, 1], [2, 1, 1], [2, 1, 2], [4, 2, 3], [5, 2, 4], [5, 3, 5], [8, 4, 7]]
[[1, 0, 0], [1, 1, 1], [2, 1, 1], [4, 2, 3]]
[[1, 1, 1], [2, 1, 2], [2, 2, 2], [2, 3, 2]]
[[1, 0, 0], [2, 1, 2], [5, 2, 4]]
[[1, 0, 0], [2, 1, 2], [4, 1, 2]]
[[1, 0, 0], [1, 1, 1], [2, 1, 1], [2, 2, 1]]

[[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [4, 1, 1], [3, 1, 1], [3, 1, 2], [4, 1, 2], [1, 2, 1], [2, 2, 1], [2, 2, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]]
[[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [4, 1, 1], [3, 1, 1], [2, 1, 2], [3, 1, 2], [1, 2, 1], [2, 2, 1], [3, 2, 3]]
[[1, 0, 0], [0, 1, 0], [2, 1, 1], [3, 1, 2], [1, 2, 1], [3, 2, 3], [4, 2, 4], [5, 3, 5], [6, 3, 6]]
[[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [1, 1, 1], [3, 1, 2], [4, 1, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]]
[[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [1, 1, 1], [3, 1, 2], [3, 2, 3], [4, 2, 4], [5, 2, 4], [6, 3, 6]]
[[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [4, 1, 1], [3, 1, 1], [2, 1, 2], [3, 1, 2

In [113]:
sg2.gaps

[[1, 0, 0], [1, 1, 1], [2, 1, 1], [2, 1, 2], [2, 2, 1], [2, 2, 2], [2, 3, 2], 
[4, 1, 2], [4, 2, 3], [5, 2, 4], [5, 3, 5], [8, 4, 7]]

In [106]:
for x in desc2:
    print(x.IsIrreducible())

True
True
True
True
True
True


In [15]:
csg1 =  Csemigroup([[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [4, 1, 1], [3, 1, 1], [3, 1, 2], [4, 1, 2], [1, 2, 1], [2, 2, 1], [2, 2, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]])
csg2 =  Csemigroup([[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [3, 1, 1], [3, 1, 2], [4, 1, 2], [1, 2, 1], [2, 2, 2], [3, 2, 3], [4, 2, 4], [5, 2, 4], [5, 3, 5], [6, 3, 6]])
csg3 =  Csemigroup([[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [4, 1, 1], [3, 1, 1], [2, 1, 2], [3, 1, 2], [1, 2, 1], [2, 2, 1], [3, 2, 3]])
csg4 =  Csemigroup([[1, 0, 0], [0, 1, 0], [2, 1, 1], [3, 1, 2], [1, 2, 1], [3, 2, 3], [4, 2, 4], [5, 3, 5], [6, 3, 6]])
csg5 =  Csemigroup([[1, 0, 0], [0, 1, 0], [2, 1, 1], [3, 1, 2], [1, 2, 1], [2, 2, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]])
csg6 =  Csemigroup([[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [1, 1, 1], [3, 1, 2], [4, 1, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]])
csg7 =  Csemigroup([[1, 0, 0], [0, 1, 0], [2, 1, 1], [3, 1, 2], [1, 2, 1], [3, 2, 3], [4, 2, 4], [2, 3, 2], [5, 3, 5], [6, 3, 6]])
csg8 =  Csemigroup([[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [1, 1, 1], [3, 1, 2], [3, 2, 3], [4, 2, 4], [5, 2, 4], [6, 3, 6]])
csg9 =  Csemigroup([[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [1, 1, 1], [2, 1, 2], [3, 1, 2]])
csg10 = Csemigroup([[1, 0, 0], [0, 1, 0], [1, 1, 1], [3, 1, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]])
csg11 = Csemigroup([[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [1, 1, 1], [2, 1, 2], [3, 1, 2]])
csg12 = Csemigroup([[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [4, 1, 1], [3, 1, 1], [2, 1, 2], [3, 1, 2], [1, 2, 1], [3, 2, 3], [4, 2, 3]])
csg13 = Csemigroup([[1, 0, 0], [0, 1, 0], [2, 1, 1], [2, 1, 2], [1, 2, 1], [3, 2, 3]])
csg14 = Csemigroup([[1, 0, 0], [0, 1, 0], [3, 1, 1], [2, 1, 2], [1, 2, 1], [3, 2, 3]])

In [122]:
iniciales = [csg1,csg2,csg3,csg4,csg5,csg6,csg7,csg8,csg9,csg10,csg11,csg12,csg13,csg14]
for x in iniciales:
    print(x.generadores,"<->",x.gaps)
    print()

[[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [4, 1, 1], [3, 1, 1], [4, 1, 2], [3, 1, 2], [1, 2, 1], [2, 2, 1], [2, 2, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]] <-> [[1, 0, 0], [1, 1, 1], [2, 1, 1], [2, 1, 2], [4, 2, 3], [5, 2, 4], [5, 3, 5], [8, 4, 7]]

[[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [3, 1, 1], [4, 1, 2], [3, 1, 2], [1, 2, 1], [2, 2, 2], [3, 2, 3], [5, 2, 4], [4, 2, 4], [5, 3, 5], [6, 3, 6]] <-> [[1, 0, 0], [1, 1, 1], [2, 1, 2], [4, 2, 3]]

[[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [4, 1, 1], [3, 1, 1], [2, 1, 2], [3, 1, 2], [1, 2, 1], [2, 2, 1], [3, 2, 3]] <-> [[1, 0, 0], [1, 1, 1], [2, 1, 1], [4, 2, 3]]

[[1, 0, 0], [0, 1, 0], [2, 1, 1], [3, 1, 2], [1, 2, 1], [3, 2, 3], [4, 2, 4], [5, 3, 5], [6, 3, 6]] <-> [[1, 1, 1], [2, 1, 2], [2, 2, 2], [2, 3, 2]]

[[1, 0, 0], [0, 1, 0], [2, 1, 1], [3, 1, 2], [1, 2, 1], [2, 2, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]] <-> [[1, 1, 1], [2, 1, 2], [5, 3, 5]]

[[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [1, 1, 1], [4, 1, 

In [ ]:
[[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [4, 1, 1], [3, 1, 1], [4, 1, 2], [3, 1, 2], [1, 2, 1], [2, 2, 1], [2, 2, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]] <-> [[1, 0, 0], [1, 1, 1], [2, 1, 1], [2, 1, 2], [4, 2, 3], [5, 2, 4], [5, 3, 5], [8, 4, 7]]

[[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [3, 1, 1], [4, 1, 2], [3, 1, 2], [1, 2, 1], [2, 2, 2], [3, 2, 3], [5, 2, 4], [4, 2, 4], [5, 3, 5], [6, 3, 6]] <-> [[1, 0, 0], [1, 1, 1], [2, 1, 2], [4, 2, 3]]

[[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [4, 1, 1], [3, 1, 1], [2, 1, 2], [3, 1, 2], [1, 2, 1], [2, 2, 1], [3, 2, 3]] <-> [[1, 0, 0], [1, 1, 1], [2, 1, 1], [4, 2, 3]]

[[1, 0, 0], [0, 1, 0], [2, 1, 1], [3, 1, 2], [1, 2, 1], [3, 2, 3], [4, 2, 4], [5, 3, 5], [6, 3, 6]] <-> [[1, 1, 1], [2, 1, 2], [2, 2, 2], [2, 3, 2]]

[[1, 0, 0], [0, 1, 0], [2, 1, 1], [3, 1, 2], [1, 2, 1], [2, 2, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]] <-> [[1, 1, 1], [2, 1, 2], [5, 3, 5]]

[[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [1, 1, 1], [4, 1, 2], [3, 1, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]] <-> [[1, 0, 0], [2, 1, 2], [5, 2, 4]]

[[1, 0, 0], [0, 1, 0], [2, 1, 1], [3, 1, 2], [1, 2, 1], [3, 2, 3], [4, 2, 4], [2, 3, 2], [5, 3, 5], [6, 3, 6]] <-> [[1, 1, 1], [2, 1, 2], [2, 2, 2]]

[[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [1, 1, 1], [3, 1, 2], [3, 2, 3], [5, 2, 4], [4, 2, 4], [6, 3, 6]] <-> [[1, 0, 0], [2, 1, 2], [4, 1, 2]]

[[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [1, 1, 1], [2, 1, 2], [3, 1, 2]] <-> [[1, 0, 0], [2, 1, 1]]

[[1, 0, 0], [0, 1, 0], [1, 1, 1], [3, 1, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]] <-> [[2, 1, 2]]

[[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [1, 1, 1], [2, 1, 2], [3, 1, 2]] <-> [[1, 0, 0]]

[[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [4, 1, 1], [3, 1, 1], [2, 1, 2], [3, 1, 2], [1, 2, 1], [3, 2, 3], [4, 2, 3]] <-> [[1, 0, 0], [1, 1, 1], [2, 1, 1], [2, 2, 1]]

[[1, 0, 0], [0, 1, 0], [2, 1, 1], [2, 1, 2], [1, 2, 1], [3, 2, 3]] <-> [[1, 1, 1]]

[[1, 0, 0], [0, 1, 0], [3, 1, 1], [2, 1, 2], [1, 2, 1], [3, 2, 3]] <-> [[1, 1, 1], [2, 1, 1]]



In [20]:
iniciales2 = [csg1,csg3,csg4,csg6,csg8,csg12]
huecos = []
for x in iniciales2:
    huecos = huecos + x.gaps
huecos = DeleteDuplicates(huecos)
print(len(huecos))
for x in huecos:
    if x not in sg2.gaps:
        print(x)

12


In [18]:
len(sg2.gaps)

12

In [35]:
for x in iniciales2:
    print(" -> ",x.generadores)
    print()

 ->  [[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [4, 1, 1], [3, 1, 1], [4, 1, 2], [3, 1, 2], [1, 2, 1], [2, 2, 1], [2, 2, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]]

 ->  [[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [3, 1, 1], [4, 1, 2], [3, 1, 2], [1, 2, 1], [2, 2, 2], [3, 2, 3], [5, 2, 4], [4, 2, 4], [5, 3, 5], [6, 3, 6]]

 ->  [[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [4, 1, 1], [3, 1, 1], [2, 1, 2], [3, 1, 2], [1, 2, 1], [2, 2, 1], [3, 2, 3]]

 ->  [[1, 0, 0], [0, 1, 0], [2, 1, 1], [3, 1, 2], [1, 2, 1], [3, 2, 3], [4, 2, 4], [5, 3, 5], [6, 3, 6]]

 ->  [[1, 0, 0], [0, 1, 0], [2, 1, 1], [3, 1, 2], [1, 2, 1], [2, 2, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]]

 ->  [[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [1, 1, 1], [4, 1, 2], [3, 1, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]]

 ->  [[1, 0, 0], [0, 1, 0], [2, 1, 1], [3, 1, 2], [1, 2, 1], [3, 2, 3], [4, 2, 4], [2, 3, 2], [5, 3, 5], [6, 3, 6]]

 ->  [[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [1, 1, 1], [3, 1, 2],

In [26]:
sg2.gaps

[[1, 0, 0], [1, 1, 1], [2, 1, 1], [2, 1, 2], [2, 2, 1], [2, 2, 2], [2, 3, 2], 
[4, 1, 2], [4, 2, 3], [5, 2, 4], [5, 3, 5], [8, 4, 7]]

In [ ]:


[[1, 0, 0], [2, 1, 1]]
[[2, 1, 2]]
[[1, 0, 0]]
[[1, 1, 1]]


In [153]:
semigrupos = [csg1,csg2,csg3,csg4,csg5,csg6,csg7,csg8,csg9,csg10,csg11,csg12,csg13,csg14]
for x in semigrupos:
    for y in semigrupos:
        if x.IsSubsemigroup(y):
            semigrupos.remove(y)
print(semigrupos)

[<__main__.Csemigroup object at 0x7f0dc684e970>, <__main__.Csemigroup object at 0x7f0dc686bfd0>, <__main__.Csemigroup object at 0x7f0dc686b790>, <__main__.Csemigroup object at 0x7f0dc686baf0>]


In [154]:
for x in semigrupos:
    print(x.generadores)

[[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [1, 1, 1], [4, 1, 2], [3, 1, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]]
[[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [1, 1, 1], [2, 1, 2], [3, 1, 2]]
[[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1], [1, 1, 1], [2, 1, 2], [3, 1, 2]]
[[1, 0, 0], [0, 1, 0], [2, 1, 1], [2, 1, 2], [1, 2, 1], [3, 2, 3]]


In [43]:
alb=[[[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [4, 1, 1],
    [3, 1, 1], [4, 1, 2], [3, 1, 2], [1, 2, 1], [2, 2, 1],
    [2, 2, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]],
    [[1, 0, 0], [0, 1, 0], [2, 1, 1], [3, 1, 2], [1, 2, 1],
    [2, 2, 2], [3, 2, 3], [4, 2, 4], [6, 3, 6]],
    [[1, 0, 0], [0, 1, 0], [1, 1, 1], [3, 1, 2], [3, 2, 3],
    [4, 2, 4], [6, 3, 6]], [[1, 0, 0], [0, 1, 0], [2, 1, 1],
    [2, 1, 2], [1, 2, 1], [3, 2, 3]], [[3, 0, 0], [2, 0, 0],
    [1, 1, 0], [0, 1, 0], [2, 1, 1], [1, 1, 1], [2, 1, 2],
    [3, 1, 2]],
    [[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1],
    [1, 1, 1], [4, 1, 2], [3, 1, 2], [3, 2, 3], [4, 2, 4],
    [6, 3, 6]],
    [[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1],
    [3, 1, 1], [4, 1, 2], [3, 1, 2], [1, 2, 1], [2, 2, 2],
    [3, 2, 3], [5, 2, 4], [4, 2, 4], [5, 3, 5], [6, 3, 6]],
    [[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [2, 1, 1],
    [1, 1, 1], [3, 1, 2], [3, 2, 3], [5, 2, 4], [4, 2, 4],
    [6, 3, 6]],
    [[1, 0, 0], [0, 1, 0], [3, 1, 1], [2, 1, 2], [1, 2, 1],
    [3, 2, 3]],
    [[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [1, 1, 1],
    [2, 1, 2], [3, 1, 2]],
    [[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [4, 1, 1],
    [3, 1, 1], [2, 1, 2], [3, 1, 2], [1, 2, 1], [2, 2, 1],
    [3, 2, 3]],
    [[1, 0, 0], [0, 1, 0], [2, 1, 1], [3, 1, 2], [1, 2, 1],
    [3, 2, 3], [4, 2, 4], [2, 3, 2], [5, 3, 5], [6, 3, 6]],
    [[1, 0, 0], [0, 1, 0], [2, 1, 1], [3, 1, 2], [1, 2, 1],
    [3, 2, 3], [4, 2, 4], [5, 3, 5], [6, 3, 6]],
    [[3, 0, 0], [2, 0, 0], [1, 1, 0], [0, 1, 0], [4, 1, 1],
    [3, 1, 1], [2, 1, 2], [3, 1, 2], [1, 2, 1], [3, 2, 3],
    [4, 2, 3]]]

In [44]:
len(alb)

14

## Corregir ejemplo

In [43]:
sg=[[2, 0, 0],[4, 2, 4],[0, 1, 0],[3, 0, 0],[6, 3, 6],[3, 1, 1],[4, 1, 1],
[3, 1, 2],[1, 1, 0],[3, 2, 3],[1, 2, 1]]

In [44]:
ss = Csemigroup(sg)

In [ ]:
lIrr=ss.DecomposeIrreducible()
for x in lIrr:
    print(x.gaps)

In [46]:
len(lIrr)

4

In [47]:
for x in lIrr:
    print(x.gaps)

[[1, 0, 0], [1, 1, 1], [2, 1, 1], [2, 1, 2], [4, 2, 3], [5, 2, 4], [5, 3, 5], [8, 4, 7]]
[[1, 1, 1], [2, 1, 2], [2, 2, 2], [2, 3, 2]]
[[1, 0, 0], [2, 1, 2], [4, 1, 2]]
[[1, 0, 0], [1, 1, 1], [2, 1, 1], [2, 2, 1]]


In [28]:
lIrr[1].IsSubsemigroup(lIrr[0])

True

In [30]:
irreduciblesNecesarios = []
for x in lIrr:
    necesario = True
    for y in lIrr:
        if x.IsSubsemigroup(y) and x != y:
            necesario = False
            break
    if necesario:
        irreduciblesNecesarios.append(x)
for x in irreduciblesNecesarios:
    print(x.gaps)

[[1, 0, 0], [1, 1, 1], [2, 1, 1], [2, 1, 2], [4, 2, 3], [5, 2, 4], [5, 3, 5], [8, 4, 7]]
[[1, 1, 1], [2, 1, 2], [2, 2, 2], [2, 3, 2]]
[[1, 0, 0], [2, 1, 2], [4, 1, 2]]
[[1, 0, 0], [1, 1, 1], [2, 1, 1], [2, 2, 1]]


In [ ]:
sg = [[3, 1], [7, 1], [8, 2], [9, 1], [9, 2], [10, 1], [10, 3], [10, 4], [11, 2], [13, 1], [14, 5], [15, 6], [17, 7], [19, 8], [21,2], [24, 2], [25, 2], [26, 11], [27, 2], [28, 2], [29, 2], [44, 3], [45, 3], [61, 4], [62, 4], [76, 5], [77, 5], [93, 6], [108, 7], [43, 3], [59, 4], [60, 4], [92, 6], [42, 18], [56, 24], [63, 27], [70, 30]]
Csg = Csemigroup(sg)
huecos = Csg.GetGaps()
f = open("huecos.txt", "a")
f.write(huecos)
f.close()

In [ ]:
len(huecos)

In [6]:
ray  = [7,3]
generators = [[3, 1], [7, 1], [8, 2], [9, 1], [9, 2], [10, 1], [10, 3], [10, 4], [11, 2], [13, 1], [14, 5], [15, 6], [17, 7], [19, 8], [21, 2], [24, 2], [25, 2], [26, 11], [27, 2], [28, 2], [29, 2], [44, 3], [45, 3], [61, 4], [62, 4], [76, 5], [77, 5], [93, 6], [108, 7], [43, 3], [59, 4], [60, 4], [92, 6], [42, 18], [56, 24], [63, 27], [70, 30]]
aux.MultiplicityAxis(ray,generators)

[42, 18]